# Generative AI & Small Language Models (SLMs)

Understanding the basics of GenAI and where SLMs fit into the hierarchy. Guide: [GenAI Basics](/slmhub/docs/learn/fundamentals/genai-basics).

## 1. The AI Hierarchy
Where do SLMs fit? Let's visualize.

In [ ]:
print("""
AI (Artificial Intelligence)
├── Machine Learning (ML)
│   ├── Deep Learning (DL)
│   │   ├── Generative AI (GenAI)
│   │   │   ├── LLMs (GPT-4, Claude) -> Cloud, Huge, $$$$
│   │   │   └── SLMs (Phi-3, Llama-3-8B) -> Local, Fast, Free <--- YOU ARE HERE
""")

## 2. Token Probability Demo
SLMs are just 'next token predictors'. Let's see this in action using a small model.

In [ ]:
!pip install transformers torch accelerate

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load a very small model for demonstration (GPT-2 is small/fast)
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

text = "The sky is"
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    predictions = outputs.logits[0, -1, :]
    probs = torch.softmax(predictions, dim=-1)

# Get top 5 predictions
top_k = 5
top_probs, top_indices = torch.topk(probs, top_k)

print(f"Input: '{text}' ... Predicting next word:\n")
for i in range(top_k):
    word = tokenizer.decode(top_indices[i])
    prob = top_probs[i].item()
    print(f"{word.strip():<10} : {prob:.2%}")

## 3. Temperature
Temperature controls randomness. Low temp = focused. High temp = creative.

In [ ]:
def generate(text, temp):
    print(f"--- Temperature: {temp} ---")
    out = model.generate(**tokenizer(text, return_tensors="pt"), 
                         max_new_tokens=20, 
                         temperature=temp, 
                         do_sample=True, 
                         top_k=50)
    print(tokenizer.decode(out[0], skip_special_tokens=True) + "\n")

text = "The confusing thing about time travel is"
generate(text, 0.1) # Focused
generate(text, 1.5) # Crazy